# Réseaux de neurones LSTM appliqué à la prédiction de grandeurs physiques (Projet NeoCampus)

 
## Résumé

Nous appliquons un réseau de neurones avec couche LSTM à la prédiction des valeurs futures des capteurs disséminés dans quelques salles du campus de l'UPS, à partir de séries temporelles des valeurs de ces mêmes capteurs. La bibliothèque keras est utilisée.

## Introduction

Dans le cadre du M1 DC, nous avons accès aux données des salles équipées de capteurs NeoCampus, et sommes chargés de la réalisation d'un modèle prédisant le niveau de confort futur. Après exploration des données, nous avons décidé d'utiliser des réseaux de neurones récurrents, qui possède une mémoire leur permettant de prendre en compte le passé. Nous avons également exploré différentes métriques d'évaluation des modèles, et comparé notre modèle avec une régression linéaire.

## Données


### Prétraitement

#### Données brutes

Des capteurs de quatre types (co2, luminosité, humidité et température) sont disposés sur six ilots. Nous disposons des valeurs enregistrées à intervalles variables (entre une seconde et une minute selon le capteur).

|type|room|value|date|unit|ilot|
|---|---|---|---|---|---|
|temperature|u4/302/temperature/ilot1|22.93|2017-09-11T12:53:14.745366|celsius|ilot1|
|temperature|u4/302/temperature/ilot2|24.16|2017-09-11T12:53:14.767362|celsius|ilot2|
|co2|u4/302/co2/ilot1|357|2017-09-11T12:53:14.794131|ppm|ilot1|
|temperature|u4/302/temperature/ilot3|24.36|2017-09-11T12:53:14.797038|celsius|ilot3|
|co2|u4/302/co2/ilot2|361|2017-09-11T12:53:14.818348|ppm|ilot2|

*Table 1 : Données brutes*

#### Prétraitement - phase 1

Les valeurs sont sélectionnées par type (ici la luminosité), agrégées par moyenne sur un interval donné (ici 15 minutes), et associées à leur valeur de confort

|timestamp|lum|confort|
|---|---|---|
|1254272|839.35|5|
|1254273|867.5833333333334|5|
|1254274|791.0666666666667|5|
|1254275|466.53333333333336|2|
|1254276|698.5932203389831|5|

*Table 2 : Luminosité de l'ilot1, moyenne sur 15 minutes*

#### Prétraitement - phase 2

Les valeurs sont regroupées en séries temporelles, puis normalisées.
Soient :

  - $v_i$ la valeur d'un capteur d'un type donné à l'instant $i$
  - $w$ la taille de la fenêtre
  - $k$ l'avance (on prédit $k$ instants après)

Nous formons un vecteur $(v_1, v_2, ..., v_w)$ que nous étiquetons avec la valeur moyenne de la $k$-ième prochaine fenêtre $\sum_{i=1}^{k}{v_{w k+i}} \over w$ :

| $v_1$ | $v_1$ | $v_1$ | $v_1$ | $\sum_{i=1}^{k}{v_{w k+i}} / w$ |
|---|---|---|---|---|
|361.11666666666673|351.8|334.31666666666666|237.56666666666663|235.1125|
|236.4|234.11864406779662|236.0|233.36666666666667|256.45296610169487|
|234.55|237.38333333333333|234.5|234.01666666666668|737.2541666666667|
|234.16666666666663|235.51666666666668|237.41666666666663|318.7118644067797|378.575|
|827.5166666666668|1219.5333333333333|531.4166666666666|370.55|351.275|

*Table 3 : Fenêtre temporelle de 4 valeurs, prédiction à i+8*

### Visualisations des données

#### Distributions

<table style="border: '0'">
        <col width="70%">
        <col>
        <!--<tr style="background: None"><td></td><td></td></td>-->
        <tr> 
            <td> <img src="images/distribution_time.png"> </td>
            <td>
                <p>La présence de longues périodes sans données recueillies (ou très peu) demandera d'être prise en compte lors de la création des séries temporelles, sans quoi certaines n'auront pas le nombre requis de mesures.</p>
            </td>
        </tr>
        <tr>
            <td> <img src="images/distributions_sensors.png"> </td>
            <td>
                <p>Ici, on peut observer la distribution des différentes colonnes des données : certains capteurs ont une large plage de valeurs possibles  distribuées de manière relativement uniforme (humidité, température), d'autres sont disposées sur des modes avec quelques occurences des valeurs non modales (co2, luminosité).
On observe que le confort ne descend que très rarement en dessous de 2 : soit la fonction de confort est mal paramétrée, soit les conditions environnementales des salles ne sont jamais inconfortables (la seconde hypothèse semble plus probable).</p>
            </td>
        </tr>
</table>

#### Corrélations

<table>
        <col width="70%">
        <col>
        <!--<tr style="background: None"><td></td><td></td></td>-->
        <tr> 
            <td> <img src="images/correlations_heatmap.png"> </td>
            <td>
                <p>Cette matrice présente la corrélation entre chaque capteur. On y observe le fonctionnement de la fonction de confort : le principal facteur est la température, suivie de la luminosité, puis de l'humidité et du co2. On remarque de relativement fortes corrélations entre luminosité et température (le soleil serait-il le facteur commun ?), et entre co2 et température/luminosité (le fait que les humains sont diurnes serait-il le facteur commun ?). L'humidité ne semble pas corrélée à quoi que ce soit de manière notable : peut-être pourrait on la retirer sans trop de pertes de performances.
On déduit de cette matrice que l'on obtiendra de biens meilleures prédictions avec tous les capteurs qu'avec un seul.</p>
            </td>
        </tr>
        <tr>
            <td> <img src="images/autocorrelation_lum.png"> </td>
            <td>
                <p>On remarque très bien la présence d'un cycle de 24h dans l'autocorrélation du capteur de luminosité. Cela permet de confirmer que les valeurs des capteurs dépendent du moment de la journée, et sont prédictibles à partir des valeurs antérieures (pas complétement, sinon on aurait une autocorrélation de 1). Cette autocorrélation diminue avec l'augmentation du délai, à cause de l'influence de paramètres externes (il est possible que demain soit plus nuageux qu'aujourd'hui...).
La luminosité présente l'autocorrélation la plus forte, mais tous les capteurs sont un minimum autocorrélés (sur des multiples de 24h).</p>
            </td>
        </tr>
</table>

#### k-means

<table>
        <col width="70%">
        <col>
        <!--<tr style="background: None"><td></td><td></td></td>-->
        <tr> 
            <td> <img src="images/kmeans_co2.png"> </td>
            <td>
                <p>L'algorithme de clustering k-means peut également être appliquée sur des séries temporelles. Les clusters seront les séries moyennes types du jeu de données.
Les lignes rouges représentent les clusters et les lignes noires (ou tâches noires en fond) les observations assignées au cluster. L'opacité des observations assignées montrent leur nombre.</p>
            </td>
        </tr>
        <tr>
            <td> <img src="images/kmeans_confort.png"> </td>
            <td>
                <p>Les périodes de changement (clusters 1 et 2) sont moins fréquentes que les périodes stables (cluster 3). Il est important de prendre cette répartition en compte afin de ne pas obtenir un modèle prédisant des séries stables (qui pourrait se voir attribuer une bonne performance selon la métrique dûe à leur forte proportion dans les données).</p>
            </td>
        </tr>
</table>


#### Décomposition saisonale

<table>
        <col width="70%">
        <col>
        <tr> 
            <td> <img src="images/decomposition_seasonal_temp.png"> </td>
            <td>
                <p>Il est possible de décomposer une série temporelle. Nous avons utilisé une décomposition saisonale naïve (statsmodels.api.tsa.seasonal_decompose()) qui permet d'obtenir les composantes "trend" (tendance), "seasonal" (saisonale/périodique) et "residual" (résidus). En combinant ces deux composantes, on obtient la courbe d'origine, aux résidus près (plus il y a de résidus, moins la décomposition permet de reproduire la série originelle : la série est bruitée (ou notre modèle inadapté)).</p>
            </td>
        </tr>
        <tr>
            <td> <img src="images/decomposition_predictions_temp.png"> </td>
            <td>
                <p>Ce graphique présente le modèle obtenu par décomposition sur une période appliquée à cette même période (en vert), puis aux restes des données (en rouge). Le comportement de la courbe n'est pas du tout le même entre ces deux périodes. Il est nécessaire de prendre en compte ces différences dans la distribution des cibles lors de l'entraînement d'un modèle (sinon on a un modèle qui prédit très bien la semaine 38 de l'année 2017, mais rien d'autre).
Ici, le modèle se trompe complètement, mais pour certains capteurs (luminosité notament), la composante saisonale reste similaire quelque soit la période de l'année.</p>
            </td>
        </tr>
</table>




## Modèle

Notre architecture de modèle est composée d'une couche d'entrées de la taille du nombre de valeurs dans une série.
Puisque le jeu de données est une série temporelle, nous avons fait le choix d'utiliser un réseau récurrent (RNN). La première couche cachée est une couche LSTM (qui se compose ici de 64 unités).

Une seconde couche cachée de 16 neurones à activation "ReLU" est utilisée pour faire une première regression linéaire et enfin un neurone de sortie à activation "sigmoid". Beaucoup de nos choix (comme la taille du batch) ont été fait de manière empirique puisque nous n'avons ni les bases théoriques ni l'expérience pratique nécessaire à la conception rigoureuse d'une architecture de modèles.

L'entraînement du modèle prend environ 3 minutes pour une vingtaine de cycles d'apprentissage ce qui a ralenti l'expérimentation ainsi que la réalisation de tous les tests que nous voulions faire. Pour améliorer les performances de l'entrainement, nous avons utilisé certains des "callbacks" de Keras tels que "ReduceLRonPlateau" et "ModelCheckpoint", qui permet d'enregistrer le meilleur modèle actuellement calculé.

L'enregistrement d'un modèle se fait sous le format "Hierarchical Data" (.h5) utilisé pour les fichiers contenant un important volume de données.

Pour la prédiction de la valeur de confort (fichier test_val_conf.py) :
Le programme charge les modèles entrainés des 4 features, predit leurs valeurs futures et calcule l'indicateur de confort via le script "annotationUnique.py" (créé par les M2)
(on a essayé de reformuler ça sans succès)

![Image du modèle non trouvée](model_plot.png "model plot")

## Résultats

Nous effectuons plusieurs tests pour estimer notre modèle, en le comparant avec la régression linéaire classique de scikit-learn.

Nous avons effectué les tests sur les données de l'ilot1, pour chaque type de capteur (courbes d'allure différentes), pour les configurations suivantes : 

 - Fenêtre de taille 2, prédiction de la fenêtre suivante
 - Fenêtre de taille 16, prédiction de la 4ème fenêtre suivante
 - Fenêtre de taille 32, prédiction de la 16ème fenêtre suivante

Lorsqu'on prédit loin dans l'avenir, aucune prédiction intermédiaire n'est effectuée.

Nous avons utilisé plusieurs métriques :

 - Mean absolute error et mean squared error : montre l'écart entre les prévisions et la réalité (l'écart entre la MAE au carré et la MSE indique la variance des erreurs). La MSE pénalise les très mauvaises prédictions. Ce sont des mesures de régression, faites sur les valeurs prédites pour le capteur.
 - Accuracy f-measure jaccard : les trois donnent exactement le même résultat, sans que nous ne sachions vraiment pourquoi :-( Ce sont des métriques de classification calculées sur le score de confort, recalculé depuis la valeur prédite du capteur.
 - Distance au sens de la DTW : probablement très adaptée à la tâche, mais trop long à calculer.

{Les performances du modèle basé sur une régression linéaire sont très proches de celle du modèle basé NN/LSTM. Le temps d'entraînement est cependant incomparable (régression : 1sec, LSTM : 2 minutes).}


Mesures de MAE, MSE et Accuracy pour les différentes combinaisons de test : les mêmes erreurs sur les mêmes tests !


| Mean average error | Mean squared error | Accuracy |
|---|---|---|
| ![mae.png](images/evaluation/mae.png) | ![mse.png](images/evaluation/mse.png) | ![accu.png](images/evaluation/accu.png) |



Nous avons réservé quelques points (début Septembre) afin de visualiser les prédictions, sur une période contigüe. La distribution de ces points est différente de celle des données d'entraînement/validation/test. Cela fausse la qualité des prédictions, mais on peut tout de même visualiser l'allure des prédictions :

|Jeu de test|MSE|Précision|Visualisation LSTM64-ReLU16-Sigm1|Visualisation régression linéaire|
|---|---|---|---|---|
|ilot1.temp.300.2.1| Faible 0.005 | Élevée 0.93 | ![contiguous.ilot1.temp.300.2.1.LSTM64-ReLU16-Sigm1.png](images/evaluation/contiguous.ilot1.temp.300.2.1.LSTM64-ReLU16-Sigm1.png) | ![contiguous.ilot1.temp.300.2.1.LinearRegression.png](images/evaluation/contiguous.ilot1.temp.300.2.1.LinearRegression.png) |
|ilot1.co2.3600.16.4 | Élevée 0.09 | Élevée 0.92 | ![contiguous.ilot1.co2.3600.16.4.LSTM64-ReLU16-Sigm1.png](images/evaluation/contiguous.ilot1.co2.3600.16.4.LSTM64-ReLU16-Sigm1.png) | ![contiguous.ilot1.co2.3600.16.4.LinearRegression.png](images/evaluation/contiguous.ilot1.co2.3600.16.4.LinearRegression.png) |
|ilot1.temp.3600.32.16| Élevée 0.1 | Faible 0.22 | ![contiguous.ilot1.temp.3600.32.16.LSTM64-ReLU16-Sigm1.png](images/evaluation/contiguous.ilot1.temp.3600.32.16.LSTM64-ReLU16-Sigm1.png) | ![contiguous.ilot1.temp.3600.32.16.LinearRegression.png](images/evaluation/contiguous.ilot1.temp.3600.32.16.LinearRegression.png) |

Pour résumer :
  - Les deux modèles sont très similaires.
  - MSE et MAE sont des métriques similaires. La précision demande une discrétisation qui pénalise certaines erreurs et pas d'autres (un écart très faible peut faire passer une prédiction dans une autre classe, et donc pénaliser cette prédiction qui était proche de la valeur réelle).
  - Plus le modèle prédit loin dans le futur, plus les prédictions sont mauvaises.

## Conclusion

Notre architecture est applicable aux 4 types de capteurs, quelque soit la taille de la fenêtre et le délai de prédiction.
Nous pouvons ensuite calculer la valeur de confort sur les prédictions, grâce au module des étudiants de M2.

La performance de ces modèles n'est pas sensiblement meilleure que par régression linéaire, tout en nécessitant un temps d'entraînement comparablement très long.  

Pour la prochaine version du modèle, il serait explorer les modèles utilisant des convolutions et/ou des GRUs, ainsi que des architectures plus complexes. Nous pensons aussi à prédire des séries temporelles (et non des valeurs), et à prédire en réutilisant prédictions antérieures. 